In [1]:
from io import StringIO
import boto3
import pandas as pd
import numpy as np
from datetime import datetime

s3 = boto3.client("s3")

def list_csv_files(bucket_name, key_path):
    csv_files = []
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=key_path)
    
    for page in page_iterator:
        if 'Contents' in page:
            for content in page['Contents']:
                if content['Key'].endswith('.csv'):
                    csv_files.append(content['Key'])
    
    return csv_files

def read_csv_files_to_dataframes(bucket_name, csv_files):
    dataframes = []
    for key in csv_files:
        # Get the object from S3
        obj = s3.get_object(Bucket=bucket_name, Key=key)
        # Read the CSV file content
        data = obj['Body'].read().decode('utf-8')
        # Convert to DataFrame
        df = pd.read_csv(StringIO(data))
        dataframes.append(df)
    return dataframes

def write_csv_to_s3(df, bucket_name, output_key):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3.put_object(Bucket=bucket_name, Key=output_key, Body=csv_buffer.getvalue())
    print(f"Uploaded to s3://{bucket_name}/{output_key}")

In [2]:
bucket_name = 'niwa-water-demand-modelling'
key_path = 'TransformedOutputs/Simulation/'
target_files = list_csv_files(bucket_name, key_path)
input_files = list_csv_files(bucket_name, "Simulation/")

In [7]:
auto_ml_job_dict = {
    'NorthWellingtonMoa': 'Canvas1734649444174',
    'WellingtonLowLevel': 'Canvas1734648978161',
    'Petone': 'Canvas1733434154045',
    'WellingtonHighWestern': 'Canvas1733085655509',
    'WellingtonHighMoa': 'Canvas1733372214860',
    'NorthWellingtonPorirua': 'Canvas1733369877242',
    'Porirua': 'Canvas1733437572452',
    'Wainuiomata': 'Canvas1734649248674',
    'UpperHutt': 'Canvas1734649294393',
    'LowerHutt': 'Canvas1734649384856'
}

for key in list(auto_ml_job_dict.keys()):
    key = f"/{key}/"
    key_inputs = [e for e in input_files if key in e and 'Final_' in e and e.endswith('.csv')]
    key_files = ["/".join(e.split("/")[1:]) for e in target_files if key in e]
    unfinished = [e for e in key_inputs if e not in key_files]
    print(f"{key}: {len(key_files)}")
    # find out which input file is not covered
    print(f"{key}: {len(unfinished)} files not processed: {unfinished}")

/NorthWellingtonMoa/: 22
/NorthWellingtonMoa/: 0 files not processed: []
/WellingtonLowLevel/: 22
/WellingtonLowLevel/: 0 files not processed: []
/Petone/: 22
/Petone/: 0 files not processed: []
/WellingtonHighWestern/: 22
/WellingtonHighWestern/: 0 files not processed: []
/WellingtonHighMoa/: 22
/WellingtonHighMoa/: 0 files not processed: []
/NorthWellingtonPorirua/: 22
/NorthWellingtonPorirua/: 0 files not processed: []
/Porirua/: 22
/Porirua/: 0 files not processed: []
/Wainuiomata/: 22
/Wainuiomata/: 0 files not processed: []
/UpperHutt/: 22
/UpperHutt/: 0 files not processed: []
/LowerHutt/: 22
/LowerHutt/: 0 files not processed: []


In [11]:
y_cols = ['Lower Hutt', 'Petone',
       'Wainuiomata', 'Upper Hutt', 'Porirua', 'Wellington High Moa',
       'Wellington High Western', 'Wellington Low Level',
       'North Wellington Moa', 'North Wellington Porirua']

# find out unique experiments
experiments = [e.split("/")[2] for e in target_files]
experiments = np.unique(experiments)

# find full results
result_files = list_csv_files(bucket_name, "Simulation/results/")
result_files = [e.split("/")[-1] for e in result_files]
experiments_with_no_results = [e for e in experiments if f"{e}_full_results.csv" not in result_files]

for exp in experiments_with_no_results:
    exp_files = [e for e in target_files if exp in e]
    exp_input_files = [e for e in input_files if exp in e]
    ordered_files = []
    df_list = []
    # process by site name
    for y_col in y_cols:
        # find input file
        input_file = [e for e in exp_input_files if f"/{y_col}.csv" in e]
        # find prediction file from this experiment
        target_file = [e for e in exp_files if f"/{y_col}.csv" in e]
        if len(input_file) == 1 and len(target_file) == 1:
            df_input = read_csv_files_to_dataframes(bucket_name, input_file)[0]
            df_target = read_csv_files_to_dataframes(bucket_name, target_file)[0]
            if "replicate" in df_input.columns:
                rep_unique = df_input["replicate"].unique()
                # check if only 1 replicate
                if len(rep_unique)>1:
                    # include replicate as index
                    df = pd.concat([df_input[["Date", "replicate"]], df_target[[y_col]]], axis=1)
                    df  = df[["Date", "replicate", y_col]].set_index(["replicate", "Date"])
                else:
                    df = pd.concat([df_input[["Date"]], df_target[[y_col]]], axis=1)
                    df  = df[["Date", y_col]].set_index("Date")
            else:
                df = pd.concat([df_input[["Date"]], df_target[[y_col]]], axis=1)
                df  = df[["Date", y_col]].set_index("Date")
                
            df_list.append(df)
            print(f"{exp}: retrieved {y_col} results")
    df_ps = pd.concat(df_list, axis=1)
    output_key = f"Simulation/results/{exp}_full_results.csv"
    write_csv_to_s3(df_ps, bucket_name, output_key)

In [12]:
df_ps

,Lower Hutt,Petone,Wainuiomata,Upper Hutt,Porirua,Wellington High Moa,Wellington High Western,Wellington Low Level,North Wellington Moa,North Wellington Porirua
Date,,,,,,,,,,
1965-01-01,222.942688,243.529099,197.892685,187.889969,183.241547,158.135315,158.770782,237.192001,167.017532,180.329132
1965-01-02,253.938736,246.500885,233.087173,236.217514,210.367355,207.861237,183.617722,246.366104,197.168396,209.394897
1965-01-03,262.940369,235.708435,233.181351,256.572601,212.539764,243.069275,229.354614,252.990662,215.087708,212.159729
1965-01-04,260.078918,246.916138,215.514084,232.516953,224.953888,188.614716,188.456665,253.301010,197.677444,224.573853
1965-01-05,258.095520,255.743942,208.626541,228.963730,206.909744,310.715271,307.831177,257.110291,195.053741,205.225296
...,...,...,...,...,...,...,...,...,...,...
2099-12-27,259.689270,232.531693,234.154221,217.543518,208.565079,200.667358,215.115936,229.896149,220.224594,226.950867
2099-12-28,253.867554,235.189621,220.098846,218.062668,211.555695,177.707397,172.829590,215.414810,197.332642,204.503998
2099-12-29,245.490372,259.935089,206.625488,183.816315,208.651886,264.604523,268.092499,237.571960,190.641266,196.917297


## clean-up transformed outputs

In [22]:
s3 = boto3.client('s3')

def delete_files_with_pattern(bucket_name, prefix, suffix):
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)
    
    for page in page_iterator:
        if 'Contents' in page:
            for content in page['Contents']:
                if content['Key'].endswith(suffix):
                    print(f"Deleting {content['Key']}")
                    s3.delete_object(Bucket=bucket_name, Key=content['Key'])

bucket_name = 'niwa-water-demand-modelling'
prefix = 'TransformedOutputs/Simulation/'
suffix = '.csv.out'
delete_files_with_pattern(bucket_name, prefix, suffix)

Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_0.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_1.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_2.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_3.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_4.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_5.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_6.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_7.csv.out
Deleting TransformedOutputs/Simulation/Final_HydroClimaticFile_ACCESS-CM2_ssp370/LowerHutt/Lower Hutt_8.